XRF Compositional Data Analysis
===============================



## Introduction



Various compositional plots can be produced using the bulk rock compositional data to characterise the rocks, including 3-component ternary plots, PCA plots and biplots.



## Data



This notebook requires `notebook.org` to have been previously run such that `./local_data/df.csv` (full dataframe) `./local_data/cleaned_normalized_df.csv` (normalized to 100% when LOI is considered) exists.



In [1]:
import pandas as pd

# Load composition data.
df = pd.read_csv("./local_data/df.csv",index_col=0)
oxide_df = pd.read_csv("./local_data/cleaned_normalized_df.csv",index_col=0)

# Get list of samples.
samples = df.index

## Oxide Component Plots



Differences between the rocks can be investigated using plots of major/characteristic oxide components (that may be combined). The two oxide sets that are plotted to characterize the rock igneous protolithare:

-   AFM - total alkali, total Fe, MgO (after the AFM plot which is used to characterize metamorphic rocks)
-   Si-Fe-Mg oxides - SiO2, total Fe, MgO (the most abundant oxides)

Total Fe will be computed from titration-derived Fe-oxides.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mpltern

# Compute the combined oxides for plotting.
oxide_df["FeTot"] = df["FeO"] + 0.9*df["Fe2O3"]
oxide_df["Alkalis"] = df["Na2O"] + df["K2O"]

### AFM



In [1]:
def plot_ternary_composition(ternary_df):
    ''' Plot sample compositions described by 3-components onto a ternary plot.

    ternary_df | :pandas.DataFrame: | Dataframe of sample compositions described by 3 components, with the index comprising sample names. This dataframe doesn't need to be normalized.

    Returns: :matplotlib.axes.Axes:
    '''
    # Normalize the ternary df columns to 100 wt% (i.e. 1).
    ternary_df = ternary_df.div(ternary_df.sum(axis=1),axis=0)
    # Use new figure.
    plt.figure()
    # Initialize ternary plot.
    ax = plt.subplot(projection="ternary")
    # Iterate through the samples in the ternary composition df and plot their AFM composition data.
    for name,data in ternary_df.iterrows():
        ax.plot(*data,label=name,marker="*",markersize=10)
    return ax


# Isolate columns for AFM plotting (in the correct order).
df_afm = oxide_df[["Alkalis","FeTot","MgO"]]
# Plot AFM points onto a ternary.
ax = plot_ternary_composition(df_afm)
# Label vertices.
ax.set_tlabel("A (Na2O + K2O)")
ax.set_llabel("F (FeO + 0.9Fe2O3)")
ax.set_rlabel("M (MgO)")
# Show legend.
plt.legend()

Legend

The points all have very low total Alkali content and cluster together, which makes interpretation difficult.



### Si-Fe-Mg Oxides



In [1]:
# Isolate columns for FMS plotting (in the correct order).
df_sfm = oxide_df[["SiO2","FeTot","MgO"]]
# Plot AFM points onto a ternary.
ax = plot_ternary_composition(df_sfm)
# Label vertices.
ax.set_tlabel("S (SiO2)")
ax.set_llabel("F (FeO + 0.9Fe2O3)")
ax.set_rlabel("M (MgO)")
# Show legend.
plt.legend()
plt.show()

None

The points have significant amounts of each vertex component, but are still clustered together and so not easy to interpret. However, it appears that 06C is much more SiO2 enriched compared to the others.

See `XRF-PCA.org/.ipynb` for PCA analysis (separate notebook to avoid mixing languages/kernels), where outliers are easier to see.



## Benard et al. 2021 biplots - WARNING: POTENTIAL LICENSING ISSUES



Geochemical biplots of one oxide against another was used by \cite{Benard2021} to differentiate between mantle from different geological settings. The plots from \cite{Benard2021} (**which are licensed under CC BY-NC-ND so these derivatives can't be published?**) were first manually extracted from the article PDF, converted to SVG format. Computer readability was enhanced by ungrouping all objects, tracing each axis with a single SVG rectangle object (and making sure no other rectangle objects were present), and then grouping the contents of each axis. The axes rectange definitions are extracted for use in matplotlib plot sizing.



In [1]:
from svg_parsing import *
# Declare the filepath to the enhanced SVG file (in the manner described above).
parent_svgfile = "./local_data/Benard2021_enhanced.svg"
# Declare the number of plot columns present in the SVG file (plot collection).
n_cols = 3
# Read the SVG file into a string.
svg = parse_svg(parent_svgfile)
# Find plot groups from the SVG file (where the groups structure has been curated such that the following code works).
parent_group = find_tag_nestable("g",svg.replace(find_tag("defs",svg,False)[0],""))
parent_group = parent_group.replace(re.search("<g[\s\S]*?>",parent_group).group(0),"")
groups = find_tag("g",parent_group,False)
# Extract information on the manually-draw axes rectangles.
rect_defs = [parse_rect(find_tag("rect",group)[0]) for group in groups]
# Generate index order that sorts the rectangles row-wise then column-wise (i.e. grid ordering).
grid_sorter = gridify(rect_defs,n_cols)
# Grid order the plot groups and axes rectangles.
ordered_groups = [groups[i] for i in grid_sorter]
ordered_rects = [rect_defs[i] for i in grid_sorter]

LOI (H2O) is ignored since the plots are for the nominally anhydrous mantle. The FeTot (vs Al2O3) plot is skipped in batch plotting since most of the Fe values plot outside range. Instead, the FeTot range and plot height are doubled (which can be seen on the SVG output but not the converted PNG output that's displayed by this notebook).

-   Note: the following code block is quite specific to this Benard et al 2021 comparison.



In [1]:
import matplotlib as mpl
import os
from io import BytesIO

# Hardcoding of plotting specifications.
# Which oxides to plot against each other (y var,x var).
plot_order = [("FeTot","Al2O3"),("SiO2","Al2O3"),("SiO2","MgO"),
              ("TiO2","MgO"),("Al2O3","MgO"),("MnO","MgO")]
# Axes ranges (y var range,x var range) to match those of Benard et al 2021.
plot_limits = [([6,9],[0,4.5]),([40,46],[0,4.5]),([40,46],[36,50]),
               ([0,0.2],[36,50]),([0,4.5],[36,50]),([0.05,0.2],[36,50])]

# Class specific to plotting comparisons to Benard et al 2021.
class PlotOxides():
    def __init__(self,df):
        # Correct post-normalization silica wt% to 44 wt% to permit comparison of these wt% to Benard et al 2021.
        df = self.correct_silica(df)
        # Compute the total Fe as defined by AFM.
        df["FeTot"] = 0.9 * df["Fe2O3"] + df["FeO"]
        self.use_df = df

    def correct_silica(self,df,expected=44):
        ''' Correct only the SiO2 column in a composition dataframe to match a prescribed SiO2 wt%. This is done by setting the total of other components to 100 wt% - SiO2 wt% and renormalizing those other columns to that (sub)total.

        df | :pandas.Dataframe: | Composition dataframe with row-wise samples. Must contain only components of interest.
        expected | Numerical | Prescribed SiO2 wt%.

        Returns: :pandas.Dataframe:
        '''
        # Only the silica correction is performed since effective FeO_Tot is unaffected by Fe oxidation state.
        non_silica_df = df[df.columns.drop("SiO2")]
        # Compute the current sample-wise sums of components other than SiO2.
        non_silica_sum = non_silica_df.sum(axis=1)
        # Compute the expected sample-wise sum of components other than SiO2.
        expected_non_silica_sum = 100 - expected
        # Normalize the sample-wise sum of non-SiO2 components to expected by multiplying each cell by expected/observed sum.
        non_silica_df = non_silica_df.mul(expected_non_silica_sum/non_silica_sum,axis=0)
        # Set SiO2 wt% to expected.
        non_silica_df["SiO2"] = expected
        return non_silica_df

    def scatter(self,x_oxide,y_oxide,xlim,ylim,figsize_pt,savefig=None):
        ''' Scatterplot one oxide against another with specified plot dimensions for saving.

        x_oxide | :str: | Name of the oxide for the x axis.
        y_oxide | :str: | Name of the oxide for the y axis.
        xlim | :list:-like [<x lower>,<x upper>] | X limit specification.
        ylim | :list:-like [<y lower>,<y upper>] | Y limit specification.
        figsize_pt | :list:-like [<width>,<height>] | Size of the plot in pt.
        savefig | :str: or :BytesIO: | SVG filename or BytesIO to save the figure output into. The figure will be saved without axes or margins.

        Returns: :matplotlib.axes.Axes:
        '''
        use_df = self.use_df
        # Extract list of samples from the composition dataframe.
        samples = use_df.index
        # Get the number of samples.
        cmap = mpl.colormaps["tab10"]
        # pt to inches conversion.
        pt = 1/72
        # Initialize figure with prescribed figure size (where figsize is in inches, converted from pt).
        fig,ax = plt.subplots(figsize=(figsize_pt[0]*pt,figsize_pt[1]*pt),tight_layout={"pad":0})
        # Plot each sample's relevant composition as a point onto the figure.
        for i,(sample,row) in enumerate(use_df.iterrows()):
            ax.scatter(row[x_oxide],row[y_oxide],color=cmap(i),s=15,edgecolors="k",label=sample)
        # Set x and y axis limits.
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        # Add (small) legend.
        ax.legend(prop={"size":5})
        if savefig is not None:
            # Save the figure in SVG format without an axis if requested.
            plt.axis("off")
            plt.savefig(savefig,bbox_inches=0,pad_inches=0,transparent=True,format="svg")
        return ax

# Remove irrelevant columns (for comparison to Benard et al 2021) from the oxide composition dataframe.
cleaned_df = oxide_df.drop(["dehyd_tot","Alkalis","FeTot","H2O"],axis=1)
# Renormalize remaining columns to 100 wt%.
cleaned_df = cleaned_df.div(cleaned_df.sum(axis=1),axis=0)*100
# Close all previous plots (to avoid plot accumulation).
plt.close("all")
# Initialize specific plotting instance.
plot_oxides = PlotOxides(cleaned_df)
# Initialize list to store the bottom left corner positions (in pt) of axes from the parent SVG file.
bottom_left_corners = []
# Initialize list to store the BytesIO instances into which the plots are saved (to memory and not disk).
fig_svgs = []
# Iterate through each axis in the parent SVG file.
for i,(rect,plot_vars,plot_limit) in enumerate(zip(ordered_rects,plot_order,plot_limits)):
    # Initialize a BytesIO instance into which the plot can be saved.
    fig_svg = BytesIO()
    # Identify the dimensions of the axes.
    pt_dimension = np.array([rect["width"],rect["height"]])
    # Check if the plot is the first plot, where y var is FeTot.
    if i == 0:
        # Increase the upper y axis limit for the first FeTot plot beyond the limit specified by Benard et al 2021.
        # Do this by doubling the vertical range (without changing the lower y axis limit).
        vertical_upscale = 2
        # Increase y axis height.
        pt_dimension[1] = pt_dimension[1] * vertical_upscale
        # Moving up the upper y limit.
        plot_limit[0][1] += (vertical_upscale-1) * (plot_limit[0][1]-plot_limit[0][0])
    # Produce and save (to BytesIO) a scatter plot of the active oxide pair.
    plot_oxides.scatter(plot_vars[1],plot_vars[0],plot_limit[1],plot_limit[0],pt_dimension,fig_svg)
    # Compute and store the bottom left corner position of the active plot.
    bottom_left_corners.append((rect["x"],rect["y"]+(rect["height"]-pt_dimension[1])))
    # Store the "saved" figure.
    fig_svgs.append(fig_svg)

# Prevent plot display.
plt.close("all")

None

The plots are then placed onto the parent figure using nested svg transformations combined with alignment of the bottom left corner.



In [1]:
# Read the parent SVG file as string.
with open(parent_svgfile) as infile:
    parent_svg = infile.read()
# Iterate through the plots produced using observed data.
for bottom_left_corner,fig_svg_io,group in zip(bottom_left_corners,fig_svgs,ordered_groups):
    # Read the active BytesIO plot "save" into string (assuming utf-8 encoding).
    fig_svg_io.seek(0)
    svg = fig_svg_io.read().decode("utf-8")
    # Find the group that encompasses the entire saved plot.
    parent_group = find_tag_nestable("g",svg)
    # Wrap this plot group within an svg tag whose bottom-left corner matches that of the relevant axis in the parent SVG.
    parent_group = '<svg x="%s" y="%s">\n' % bottom_left_corner + parent_group + "\n</svg>"
    # Add the plot group to the parent SVG.
    parent_svg = parent_svg.replace(group,group.replace("</g>",parent_group+"\n</g>"))

# Save the updated parent SVG (i.e. after plots using observed data have been overlain on the parent SVG axes).
out_svg = "./imgs/Benard2021_plotted.svg"
with open(out_svg,"w") as outfile:
    outfile.write(parent_svg)

# Convert the output SVG to PNG for display (nested <svg>s appear to not be supported by IPython.display.SVG so cannot use that).
import subprocess
png_file = out_svg+".png"
subprocess.call(["inkscape",out_svg,"-o",png_file],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

from IPython.display import Image
Image(filename=png_file)

<IPython.core.display.Image object>

These plots reveal that the sample compositions are broadly consistent with an abyssal mantle source, with the notable exception of 07A, which is more consistent with an off-craton mantle source. The outlier nature of 07A is consistent with the PCA plots.

